# Web(http) proxy by python
----

**개발환경** : OS X

----
1. 자신의 웹클라이언트(client) LAN 설정을 127.0.0.1:8080으로 설정한다. web_proxy를 만들어 8080 port를 열어 놓고 client로부터 오는 HTTP request를 받아 들인다. proxy는 여기에서 Host 부분을 추출하여 실제 웹서버(server)에 접속하여 client로부터 받은 HTTP request를 server로 보내고, server로부터 받은 HTTP response를 client에게 보낸다.

2. 간단한 HTTP body의 내용을 조작하여 본다. 예를 들면 client에서 test.gilgil.net에 접속했을 때 보이는 "hacking"이라는 단어를 "ABCDEFG"로 바꾸거나, daum.net "마이클잭슨"의 검색 결과에서 "Michael"을 "GILBERT"로 바꿀 수 있도록 한다.

3. ~~web_proxy에서 간단한 file caching 기능을 추가하여, client로부터 똑같은 파일(그림이나 zip file 등)의 요청이 들어 오는 경우 caching된 컨텐츠를 이용하여 client로 바로 응답을 주도록 한다(이로 인해 server의 부하를 줄일 수 있음).~~
4. ~~HTTP뿐만 아니라 HTTPS의 통신도 web_proxy를 통해서 proxying을 해 줄 수 있도록 한다.~~
----



## [A] Simple Proxy




* 자신의 웹클라이언트(client) LAN 설정을 127.0.0.1:8080으로 설정한다.

Chrome 설정 -> 고급설정 -> 네트워크 프록시 설정 변경 or 시스템환경설정 -> 네트워크 -> 고급
(4차과제 이미지 참고)
* web_proxy를 만들어 8080 port를 열어 놓고 client로부터 오는 HTTP request를 받아 들인다.
    proxy는 여기에서 Host 부분을 추출하여 실제 웹서버(server)에 접속하여 client로부터 받은 HTTP request를 server로 보내고, server로부터 받은 HTTP response를 client에게 보낸다.

In [1]:
import sys
import multiprocessing
import datetime
import argparse
import logging
import socket
import select
import string
import zlib

In [2]:
logger = logging.getLogger(__name__)
VERSION = (0, 2)
__version__ = '.'.join(map(str, VERSION[0:2]))

text_type = unicode
binary_type = str
import urlparse

In [43]:
def text_(s, encoding='utf-8', errors='strict'):
    """ If ``s`` is an instance of ``binary_type``, return
    ``s.decode(encoding, errors)``, otherwise return ``s``"""
    if isinstance(s, binary_type):
        return s.decode(encoding, errors)
    return s  

def bytes_(s, encoding='utf-8', errors='strict'):
    """ If ``s`` is an instance of ``text_type``, return
    ``s.encode(encoding, errors)``, otherwise return ``s``"""
    if isinstance(s, text_type): 
        return s.encode(encoding, errors)
    return s

version = bytes_(__version__)

CRLF, COLON, SP = b'\r\n', b':', b' '

HTTP_REQUEST_PARSER = 1
HTTP_RESPONSE_PARSER = 2

HTTP_PARSER_STATE_INITIALIZED = 1
HTTP_PARSER_STATE_LINE_RCVD = 2
HTTP_PARSER_STATE_RCVING_HEADERS = 3
HTTP_PARSER_STATE_HEADERS_COMPLETE = 4
HTTP_PARSER_STATE_RCVING_BODY = 5
HTTP_PARSER_STATE_COMPLETE = 6

CHUNK_PARSER_STATE_WAITING_FOR_SIZE = 1
CHUNK_PARSER_STATE_WAITING_FOR_DATA = 2
CHUNK_PARSER_STATE_COMPLETE = 3
CHUNK_SIZE = 8192  #1024 error

parser

In [80]:
class ChunkParser(object):
    """
    chuncked 된 HTTP response parser.
    제대로 안됨..
    """

    def __init__(self):
        self.state = CHUNK_PARSER_STATE_WAITING_FOR_SIZE
        self.body = b''
        self.chunk = b''
        self.size = None
    
    def parse(self, data):
        
        if len(data) > 0:
            more = True 
        else:
            False
        
        while more: 
            more, data = self.process(data)
            
        #self.body 가 완성된 부분.
        z = zlib.decompressobj(16+zlib.MAX_WBITS)
        """
        http://stackoverflow.com/questions/2423866/python-decompressing-gzip-chunk-by-chunk
        error 참조
        """
        
        """
        temp = self.body
        li = []
        print "self.body len: " , len(self.body)
        n = int(len(self.body)/CHUNK_SIZE)
        print "n : ", n
        
        for k in range(0,n+1):
            if k is n:
                li.append(self.body[n:len(self.body)])
            else:
                li.append(self.body[k:CHUNK_SIZE*(k+1)])
                
                
        print "len of li : " , len(li)

        i = 0
        lilen = len(li)
        body_decomp = ""
        while True and i < lilen:
            buf = z.unconsumed_tail
            if buf == "" :      
                buf = li[i]
                print "len of buf : " , len(buf)
                i += 1
                if buf == "":
                    break
            got = z.decompress(buf)
            body_decomp += got
            if got == "":
                break
                
        body_decomp = string.replace(body_decomp, 'Michael', 'GILBERT')
        
        self.body = body_decomp
        print self.body
        """

        body_decomp = z.decompress(self.body)
        body_decomp = string.replace(body_decomp, 'Michael', 'GILBERT')
        #print body_decomp
        #self.body = body_decomp
        

    def process(self, data):
        if self.state == CHUNK_PARSER_STATE_WAITING_FOR_SIZE:
            line, data = HttpParser.split(data)
            self.size = int(line, 16)
            self.state = CHUNK_PARSER_STATE_WAITING_FOR_DATA
        elif self.state == CHUNK_PARSER_STATE_WAITING_FOR_DATA:
            remaining = self.size - len(self.chunk)
            self.chunk += data[:remaining]
            """
            정제된 data. 
            """
            data = data[remaining:]
            if len(self.chunk) == self.size:
                data = data[len(CRLF):]
                self.body += self.chunk
                if self.size == 0:
                    self.state = CHUNK_PARSER_STATE_COMPLETE
                else:
                    self.state = CHUNK_PARSER_STATE_WAITING_FOR_SIZE
                self.chunk = b''
                self.size = None

        return len(data) > 0, data
    
        
class HttpParser(object):
    """HTTP request/response parser."""
    
    def __init__(self, type=None):
        self.state = HTTP_PARSER_STATE_INITIALIZED
        self.type = type if type else HTTP_REQUEST_PARSER
        
        self.raw = b''
        self.buffer = b''
        
        self.headers = dict()
        self.body = None
        
        self.method = None
        self.url = None
        self.code = None
        self.reason = None
        self.version = None
        
        self.chunker = None
    
    def parse(self, data):
        #data -> str.
        self.raw += data
        data = self.buffer + data
        self.buffer = b''
        
        if len(data) > 0:
            more = True 
        else:
            False
        while more: 
            more, data = self.process(data)
        self.buffer = data
        

    def process(self, data):
        if self.state >= HTTP_PARSER_STATE_HEADERS_COMPLETE and \
        (self.method == b"POST" or self.type == HTTP_RESPONSE_PARSER):
            if not self.body:
                self.body = b''

            #chuncked가 아니라 content-length가 header에 있는 경우
            if b'content-length' in self.headers:
                self.state = HTTP_PARSER_STATE_RCVING_BODY
                self.body += data
                if len(self.body) >= int(self.headers[b'content-length'][1]):
                    self.state = HTTP_PARSER_STATE_COMPLETE
            #transfer-encoding이 chuncked인 경우.
            elif b'transfer-encoding' in self.headers and self.headers[b'transfer-encoding'][1].lower() == b'chunked':
                #print "Chuncked가 나왔어!!!!"
                if not self.chunker:
                    self.chunker = ChunkParser()
                self.chunker.parse(data)
                if self.chunker.state == CHUNK_PARSER_STATE_COMPLETE:
                    self.body = self.chunker.body
                    self.state = HTTP_PARSER_STATE_COMPLETE
            #charset utf-8
            if b'charset' in self.headers :# and self.headers[b'charset'][1].lower() == b'utf-8'  :
                print self.headers[b'charset'][1].lower()
                print type(self.body)
                print self.body
            
            return False, b''
        
        line, data = HttpParser.split(data)
        if line == False: return line, data
        
        if self.state < HTTP_PARSER_STATE_LINE_RCVD:
            self.process_line(line)
        elif self.state < HTTP_PARSER_STATE_HEADERS_COMPLETE:
            self.process_header(line)
        
        if self.state == HTTP_PARSER_STATE_HEADERS_COMPLETE and \
        self.type == HTTP_REQUEST_PARSER and \
        not self.method == b"POST" and \
        self.raw.endswith(CRLF*2):
            self.state = HTTP_PARSER_STATE_COMPLETE
        
        return len(data) > 0, data
    
    def process_line(self, data):
        line = data.split(SP)
        if self.type == HTTP_REQUEST_PARSER:
            self.method = line[0].upper()
            self.url = urlparse.urlsplit(line[1])
            self.version = line[2]
        else:
            self.version = line[0]
            self.code = line[1]
            self.reason = b' '.join(line[2:])
        self.state = HTTP_PARSER_STATE_LINE_RCVD
    
    def process_header(self, data):
        if len(data) == 0:
            if self.state == HTTP_PARSER_STATE_RCVING_HEADERS:
                self.state = HTTP_PARSER_STATE_HEADERS_COMPLETE
            elif self.state == HTTP_PARSER_STATE_LINE_RCVD:
                self.state = HTTP_PARSER_STATE_RCVING_HEADERS
        else:
            self.state = HTTP_PARSER_STATE_RCVING_HEADERS
            parts = data.split(COLON)
            key = parts[0].strip()
            value = COLON.join(parts[1:]).strip()
            self.headers[key.lower()] = (key, value)
    
    def build_url(self):
        if not self.url:
            return b'/None'
        
        url = self.url.path
        if url == b'': url = b'/'
        if not self.url.query == b'': url += b'?' + self.url.query
        if not self.url.fragment == b'': url += b'#' + self.url.fragment
        return url
    
    def build_header(self, k, v):
        return k + b": " + v + CRLF
    
    def build(self, del_headers=None, add_headers=None):
        req = b" ".join([self.method, self.build_url(), self.version])
        req += CRLF
        
        if not del_headers: del_headers = []
        for k in self.headers:
            if not k in del_headers:
                req += self.build_header(self.headers[k][0], self.headers[k][1])
        
        if not add_headers: add_headers = []
        for k in add_headers:
            req += self.build_header(k[0], k[1])
        
        req += CRLF
        if self.body:
            req += self.body
        
        return req
    
    @staticmethod
    def split(data):
        pos = data.find(CRLF)
        if pos == -1: return False, data
        line = data[:pos]
        data = data[pos+len(CRLF):]
        return line, data

---

Connection

In [81]:
class Connection(object):
    """TCP server/client connection abstraction."""
    
    def __init__(self, what):
        self.buffer = b''
        self.closed = False
        self.what = what # server or client
    
    def send(self, data):
        return self.conn.send(data)
    
    def recv(self, bytes=8192):
        
        try:
            data = self.conn.recv(bytes)
            if len(data) == 0:
                logger.debug('recvd 0 bytes from %s' % self.what)
                return None
            logger.debug('rcvd %d bytes from %s' % (len(data), self.what))
            
            return data
        except Exception as e:
            logger.exception('Exception while receiving from connection %s %r with reason %r' % (self.what, self.conn, e))
            return None
    
    def close(self):
        self.conn.close()
        self.closed = True
    
    def buffer_size(self):
        return len(self.buffer)
    
    def has_buffer(self):
        return self.buffer_size() > 0
    
    def queue(self, data):
        self.buffer += data
    
    def flush(self):
        sent = self.send(self.buffer)
        self.buffer = self.buffer[sent:]
        logger.debug('flushed %d bytes to %s' % (sent, self.what))

---
server,client,proxyerror, ProxyConnectionFailed

In [82]:

class Server(Connection):
    """Establish connection to destination server."""
    
    def __init__(self, host, port):
        super(Server, self).__init__(b'server')
        self.addr = (host, int(port))
    
    def connect(self):
        self.conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.conn.connect((self.addr[0], self.addr[1]))

class Client(Connection):
    """Accepted client connection."""
    
    def __init__(self, conn, addr):
        super(Client, self).__init__(b'client')
        self.conn = conn
        self.addr = addr

class ProxyError(Exception):
    pass

class ProxyConnectionFailed(ProxyError):
    
    def __init__(self, host, port, reason):
        self.host = host
        self.port = port
        self.reason = reason
    
    def __str__(self):
        return '<ProxyConnectionFailed - %s:%s - %s>' % (self.host, self.port, self.reason)

proxy

In [83]:
class Proxy(multiprocessing.Process):
    """HTTP proxy implementation.
    
    Accepts connection object and act as a proxy between client and server.
    """
    
    def __init__(self, client):
        super(Proxy, self).__init__()
        
        self.start_time = self._now()
        self.last_activity = self.start_time
        
        self.client = client
        self.server = None
        
        self.request = HttpParser()
        self.response = HttpParser(HTTP_RESPONSE_PARSER)
        
        self.connection_established_pkt = CRLF.join([
            b'HTTP/1.1 200 Connection established',
            b'Proxy-agent: proxy.py v' + version,
            CRLF
        ])
    
    def _now(self):
        return datetime.datetime.utcnow()
    
    def _inactive_for(self):
        return (self._now() - self.last_activity).seconds
    
    def _is_inactive(self):
        return self._inactive_for() > 30
    
    def _process_request(self, data):
        # once we have connection to the server
        # we don't parse the http request packets
        # any further, instead just pipe incoming
        # data from client to server
        if self.server and not self.server.closed:
            self.server.queue(data)
            return
        
        # parse http request
        self.request.parse(data)
        
        # once http request parser has reached the state complete
        # we attempt to establish connection to destination server
        if self.request.state == HTTP_PARSER_STATE_COMPLETE:
            logger.debug('request parser is in state complete')
            
            if self.request.method == b"CONNECT":
                host, port = self.request.url.path.split(COLON)
            elif self.request.url:
                host, port = self.request.url.hostname, self.request.url.port if self.request.url.port else 80
            
            self.server = Server(host, port)
            try:
                logger.debug('connecting to server %s:%s' % (host, port))
                self.server.connect()
                logger.debug('connected to server %s:%s' % (host, port))
            except Exception as e:
                self.server.closed = True
                raise ProxyConnectionFailed(host, port, repr(e))
            
            # for http connect methods (https requests)
            # queue appropriate response for client 
            # notifying about established connection
            if self.request.method == b"CONNECT":
                self.client.queue(self.connection_established_pkt)
            # for usual http requests, re-build request packet
            # and queue for the server with appropriate headers
            else:
                self.server.queue(self.request.build(
                    del_headers=[b'proxy-connection', b'connection', b'keep-alive'], 
                    add_headers=[(b'Connection', b'Close')]
                ))
    
    def _process_response(self, data):
        # parse incoming response packet
        # only for non-https requests
        if not self.request.method == b"CONNECT":
            self.response.parse(data)
        
        # queue data for client
        self.client.queue(data)
    
    def _access_log(self):
        host, port = self.server.addr if self.server else (None, None)
        if self.request.method == b"CONNECT":
            logger.info("%s:%s - %s %s:%s" % (self.client.addr[0], self.client.addr[1], self.request.method, host, port))
        elif self.request.method:
            logger.info("%s:%s - %s %s:%s%s - %s %s - %s bytes" % (self.client.addr[0], self.client.addr[1], self.request.method, host, port, self.request.build_url(), self.response.code, self.response.reason, len(self.response.raw)))
        
    def _get_waitable_lists(self):
        rlist, wlist, xlist = [self.client.conn], [], []
        logger.debug('*** watching client for read ready')
        
        if self.client.has_buffer():
            logger.debug('pending client buffer found, watching client for write ready')
            wlist.append(self.client.conn)
        
        if self.server and not self.server.closed:
            logger.debug('connection to server exists, watching server for read ready')
            rlist.append(self.server.conn)
        
        if self.server and not self.server.closed and self.server.has_buffer():
            logger.debug('connection to server exists and pending server buffer found, watching server for write ready')
            wlist.append(self.server.conn)
        
        return rlist, wlist, xlist
    
    def _process_wlist(self, w):
        if self.client.conn in w:
            logger.debug('client is ready for writes, flushing client buffer')
            self.client.flush()
        
        if self.server and not self.server.closed and self.server.conn in w:
            logger.debug('server is ready for writes, flushing server buffer')
            self.server.flush()
    
    def _process_rlist(self, r):
        if self.client.conn in r:
            logger.debug('client is ready for reads, reading')
            data = self.client.recv()
            self.last_activity = self._now()
            
            if not data:
                logger.debug('client closed connection, breaking')
                return True
            
            try:
                self._process_request(data)
            except ProxyConnectionFailed as e:
                logger.exception(e)
                self.client.queue(CRLF.join([
                    b'HTTP/1.1 502 Bad Gateway',
                    b'Proxy-agent: proxy.py v' + version,
                    b'Content-Length: 11',
                    b'Connection: close',
                    CRLF
                ]) + b'Bad Gateway')
                self.client.flush()
                return True
        
        if self.server and not self.server.closed and self.server.conn in r:
            logger.debug('server is ready for reads, reading')
            data = self.server.recv()
            self.last_activity = self._now()
            
            if not data:
                logger.debug('server closed connection')
                self.server.close()
            else:
                self._process_response(data)
        
        return False
    
    def _process(self):
        while True:
            rlist, wlist, xlist = self._get_waitable_lists()
            r, w, x = select.select(rlist, wlist, xlist, 1)
            
            self._process_wlist(w)
            if self._process_rlist(r):
                break
            
            if self.client.buffer_size() == 0:
                if self.response.state == HTTP_PARSER_STATE_COMPLETE:
                    logger.debug('client buffer is empty and response state is complete, breaking')
                    break
                
                if self._is_inactive():
                    logger.debug('client buffer is empty and maximum inactivity has reached, breaking')
                    break
    
    def run(self):
        logger.debug('Proxying connection %r at address %r' % (self.client.conn, self.client.addr))
        try:
            self._process()
        except KeyboardInterrupt:
            pass
        except Exception as e:
            logger.exception('Exception while handling connection %r with reason %r' % (self.client.conn, e))
        finally:
            logger.debug("closing client connection with pending client buffer size %d bytes" % self.client.buffer_size())
            self.client.close()
            if self.server:
                logger.debug("closed client connection with pending server buffer size %d bytes" % self.server.buffer_size())
            self._access_log()
            logger.debug('Closing proxy for connection %r at address %r' % (self.client.conn, self.client.addr))


tcp,http

In [84]:
class TCP(object):
    """TCP server implementation."""
    
    def __init__(self, hostname='127.0.0.1', port=8080, backlog=100):
        self.hostname = hostname
        self.port = port
        self.backlog = backlog
    
    def handle(self, client):
        raise NotImplementedError()
    
    def run(self):
        try:
            logger.info('Starting server on port %d' % self.port)
            self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            self.socket.bind((self.hostname, self.port))
            self.socket.listen(self.backlog)
            while True:
                conn, addr = self.socket.accept()
                logger.debug('Accepted connection %r at address %r' % (conn, addr))
                client = Client(conn, addr)
                self.handle(client)
        except Exception as e:
            logger.exception('Exception while running the server %r' % e)
        finally:
            logger.info('Closing server socket')
            self.socket.close()

class HTTP(TCP):
    """HTTP proxy server implementation.
    
    Spawns new process to proxy accepted client connection.
    """
    
    def handle(self, client):
        proc = Proxy(client)
        proc.daemon = True
        proc.start()
        logger.debug('Started process %r to handle connection %r' % (proc, client.conn))



## [B] HTTP body 조작

Connection class의 recv function에서 data자체를 조작한다.
type(data) == str
connection 재실행 후 뒷 코드 부분 부터 다시 컴파일

In [86]:
class Connection(object):
    """TCP server/client connection"""
    
    def __init__(self, what):
        self.buffer = b''
        self.closed = False
        self.what = what # server or client
    
    def send(self, data):
        return self.conn.send(data)
    
    def recv(self, bytes=8192):
        """
        recv. B 과제는 여기서 수정 가능
        data 조작. (header다 포함된 data . body만이 아님)
        """
        try:
            data = self.conn.recv(bytes)
            if len(data) == 0:
                logger.debug('recvd 0 bytes from %s' % self.what)
                return None
            logger.debug('rcvd %d bytes from %s' % (len(data), self.what))
            
            data = string.replace(data, 'hacking', 'ABCDEFG')
            
            print data
            return data
        except Exception as e:
            logger.exception('Exception while receiving from connection %s %r with reason %r' % (self.what, self.conn, e))
            return None
    
    def close(self):
        self.conn.close()
        self.closed = True
    
    def buffer_size(self):
        return len(self.buffer)
    
    def has_buffer(self):
        return self.buffer_size() > 0
    
    def queue(self, data):
        self.buffer += data
    
    def flush(self):
        sent = self.send(self.buffer)
        self.buffer = self.buffer[sent:]
        logger.debug('flushed %d bytes to %s' % (sent, self.what))

# 실행

In [85]:
hostname = '127.0.0.1'
port = 8080

try:
    proxy = HTTP(hostname, port)
    proxy.run()
except KeyboardInterrupt:
    pass


<!doctype html>
<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" class="mac chrome ">









<head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta http-equiv="content-Type" content="text/html;charset=utf-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta name="autocomplete" content="off" />
<meta property="og:title" content="마이클잭슨 &ndash; Daum 통합 검색" />
<meta property="og:url" content="http://search.daum.net/search?w=tot&q=%EB%A7%88%EC%9D%B4%ED%81%B4%EC%9E%AD%EC%8A%A8" />
<meta property="og:description" content="Daum 검색에서 마이클잭슨에 대한 최신정보를 찾아보세요." />

<meta property="og:image" content="http://m1.daumcdn.net/svc/image/U03/common_icon/557566730358050001" />

<title>마이클잭슨 &ndash; Daum 통합 검색</title>

<link rel="search" type="application/opensearchdescription+xml" href="http://top-sc.daum-img.net/w3c/OpenSearch.xml" title="Daum">

<link rel="stylesheet" type="text/css" href="//search1.daumcdn.net/search/statics/common/css/2016/p_common.1123115639.css" charset="ut

다음에서 오는 html코드가 다 깨진 상태로 str로 저장되어서 Michael검색 자체가 불가능.

In [90]:
import requests
r = requests.get('http://daum.net')
r.encoding


'UTF-8'

In [93]:
r = requests.get('http://test.gilgil.net')
r.encoding


'ISO-8859-1'

In [94]:
r = requests.get('http://ted.com')
r.encoding

'utf-8'

Chunked를 transfer-encoding으로 사용하는 사이트에서 또한 utf-8 을 charset으로 사용하는 사이트에서 실제 데이터 부분이 다 깨져서 비교가 이루어 지지 않는 상태. t


```
\r\n\x1f\x8b\x08\x00\x00\x00\x00\x00\x00
```
이런 데이터로 존재

```
z = zlib.decompressobj(16+zlib.MAX_WBITS)
body_decomp = z.decompress(self.body)
```
로 gzip decompress를 할 수 있으나 self.body수정시 오류..